#### CSC 180 Intelligent Systems 

#### California State University, Sacramento




## Lab 6: Evaluating Neural Networks 


# Helpful Functions for Tensorflow (little gems)

The following functions will be used with TensorFlow to help preprocess the data.  They allow you to build the feature vector for a neural network. 

* Predictors/Inputs 
    * Fill any missing inputs with the median for that column.  Use **missing_median**.
    * Encode textual/categorical values with **encode_text_dummy**.
    * Encode numeric values with **encode_numeric_zscore**.
* Output
    * Discard rows with missing outputs.
    * Encode textual/categorical values with **encode_text_index**.
    * Do not encode output numeric values.
* Produce final feature vectors (x) and expected output (y) with **to_xy**.

In [1]:
from collections.abc import Sequence
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column. 
    target_type = df[target].dtypes
    target_type = target_type[0] if isinstance(target_type, Sequence) else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    else:
        # Regression
        return df[result].values.astype(np.float32), df[target].values.astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low


# Training with a Test Set with Early Stopping

**Overfitting** occurs when a neural network is trained to the point that it begins to memorize rather than generalize.  

![Training vs Validation Error for Overfitting](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/class_3_training_val.png "Training vs Validation Error for Overfitting")


## Use train_test_split and earlystopping to avoid overfitting

In [2]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split

from sklearn import metrics

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

path = "./data/"
    
filename = os.path.join(path,"iris.csv")    
df = pd.read_csv(filename,na_values=['NA','?'])

species = encode_text_index(df,"species")

x,y = to_xy(df,"species")

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

model = Sequential()

model.add(Dense(10, input_dim=x.shape[1], activation='relu'))
model.add(Dense(5,activation='relu'))
model.add(Dense(y.shape[1],activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=2, verbose=2, mode='auto')  

# patience: number of epochs with no improvement after which training will be stopped

# The test set is checked during training to monitor progress for early stopping but is never used for gradient descent (model training)

model.fit(x_train, y_train, validation_data=(x_test,y_test), callbacks=[monitor], verbose=2, epochs=1000)  





Epoch 1/1000

4/4 - 1s - loss: 3.8650 - val_loss: 3.3732 - 575ms/epoch - 144ms/step
Epoch 2/1000
4/4 - 0s - loss: 3.6571 - val_loss: 3.1938 - 24ms/epoch - 6ms/step
Epoch 3/1000
4/4 - 0s - loss: 3.4593 - val_loss: 3.0186 - 23ms/epoch - 6ms/step
Epoch 4/1000
4/4 - 0s - loss: 3.2640 - val_loss: 2.8574 - 22ms/epoch - 6ms/step
Epoch 5/1000
4/4 - 0s - loss: 3.1011 - val_loss: 2.7252 - 22ms/epoch - 5ms/step
Epoch 6/1000
4/4 - 0s - loss: 2.9621 - val_loss: 2.6012 - 22ms/epoch - 6ms/step
Epoch 7/1000
4/4 - 0s - loss: 2.8215 - val_loss: 2.4815 - 22ms/epoch - 5ms/step
Epoch 8/1000
4/4 - 0s - loss: 2.6924 - val_loss: 2.3667 - 22ms/epoch - 5ms/step
Epoch 9/1000
4/4 - 0s - loss: 2.5728 - val_loss: 2.2585 - 22ms/epoch - 5ms/step
Epoch 10/1000
4/4 - 0s - loss: 2.4606 - val_loss: 2.1587 - 21ms/epoch - 5ms/step
Epoch 11/1000
4/4 - 0s - loss: 2.3479 - val_loss: 2.0687 - 22ms/epoch - 6ms/step
Epoch 12/1000
4/4 - 0s - loss: 2.2493 - val_loss: 1.9854 - 22ms/epoch - 5ms/step
Epoch 13/1000
4/4 - 0s - loss:

Now that the neural network is trained, we can make predictions about the test set.  The following code predicts the type of iris for test set and displays the first five irises. 

In [3]:
pred = model.predict(x_test)
print(pred[0:5]) # print first five predictions

2/2 [==============================] - 0s 3ms/step
[[2.3811130e-02 7.8876299e-01 1.8742588e-01]
 [9.7003436e-01 2.9365739e-02 5.9989374e-04]
 [2.6326935e-04 6.0994785e-02 9.3874192e-01]
 [2.2569831e-02 7.1530962e-01 2.6212057e-01]
 [1.9923830e-02 7.6651919e-01 2.1355698e-01]]


Each line provides the probability that the iris is one of the 3 types of iris in the data set. 

## Saving Best Weights using ModelCheckpoint

It would be good idea to keep track of the most optimal weights during the entire training operation.  


ModelCheckpoint saves a copy of the neural network to **best_weights.hdf5** each time the validation score of the neural network improves.  

Once training is done, we just reload this file and we have the optimal training weights that were found.

In [4]:
import pandas as pd
import io
import requests
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

path = "./data/"
    
filename = os.path.join(path,"iris.csv")    
df = pd.read_csv(filename,na_values=['NA','?'])

species = encode_text_index(df,"species")
x,y = to_xy(df,"species")

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], activation='relu'))
model.add(Dense(5,activation='relu'))
model.add(Dense(y.shape[1],activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

checkpointer = ModelCheckpoint(filepath="dnn/best_weights.hdf5", verbose=0, save_best_only=True) # save best model

model.fit(x_train, y_train,validation_data=(x_test,y_test),callbacks=[monitor,checkpointer],verbose=2,epochs=1000)

model.load_weights('dnn/best_weights.hdf5') # load weights from best model

Epoch 1/1000
4/4 - 0s - loss: 2.1141 - val_loss: 1.8587 - 471ms/epoch - 118ms/step
Epoch 2/1000
4/4 - 0s - loss: 2.0267 - val_loss: 1.7848 - 31ms/epoch - 8ms/step
Epoch 3/1000
4/4 - 0s - loss: 1.9464 - val_loss: 1.7155 - 31ms/epoch - 8ms/step
Epoch 4/1000
4/4 - 0s - loss: 1.8665 - val_loss: 1.6514 - 46ms/epoch - 11ms/step
Epoch 5/1000
4/4 - 0s - loss: 1.7936 - val_loss: 1.5918 - 38ms/epoch - 9ms/step
Epoch 6/1000


c:\Users\cdub6\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4/4 - 0s - loss: 1.7293 - val_loss: 1.5358 - 31ms/epoch - 8ms/step
Epoch 7/1000
4/4 - 0s - loss: 1.6638 - val_loss: 1.4843 - 32ms/epoch - 8ms/step
Epoch 8/1000
4/4 - 0s - loss: 1.6064 - val_loss: 1.4362 - 32ms/epoch - 8ms/step
Epoch 9/1000
4/4 - 0s - loss: 1.5544 - val_loss: 1.3916 - 30ms/epoch - 7ms/step
Epoch 10/1000
4/4 - 0s - loss: 1.5033 - val_loss: 1.3511 - 33ms/epoch - 8ms/step
Epoch 11/1000
4/4 - 0s - loss: 1.4558 - val_loss: 1.3145 - 32ms/epoch - 8ms/step
Epoch 12/1000
4/4 - 0s - loss: 1.4172 - val_loss: 1.2803 - 30ms/epoch - 7ms/step
Epoch 13/1000
4/4 - 0s - loss: 1.3769 - val_loss: 1.2499 - 32ms/epoch - 8ms/step
Epoch 14/1000
4/4 - 0s - loss: 1.3425 - val_loss: 1.2218 - 31ms/epoch - 8ms/step
Epoch 15/1000
4/4 - 0s - loss: 1.3109 - val_loss: 1.1962 - 32ms/epoch - 8ms/step
Epoch 16/1000
4/4 - 0s - loss: 1.2807 - val_loss: 1.1732 - 31ms/epoch - 8ms/step
Epoch 17/1000
4/4 - 0s - loss: 1.2530 - val_loss: 1.1523 - 31ms/epoch - 8ms/step
Epoch 18/1000
4/4 - 0s - loss: 1.2291 - val_l

### Potential Keras Issue on Small Networks Regarding Saving Optimal Weights

You might occasionally see this error:

```
OSError: Unable to create file (Unable to open file: name = 'dnn/best_weights.hdf5', errno = 22, error message = 'invalid argument', flags = 13, o_flags = 302)
```

Usually you can just run rerun the code and it goes away.  This is an unfortnuate result of saving a file each time the validation score improves (as described in the previous section).  If the errors improve two rapidly, you might try to save the file twice and get an error from these two saves overlapping.  For larger neural networks this will not be a problem because each training step will take longer, allowing for plenty of time for the previous save to complete.   

## Evaluating Classification Models

### (1) Calculate Classification Accuracy/Precision/Recall/F1-Score

By default, Keras will return the predicted probability for each class. We can change these prediction probabilities into the actual iris predicted with **argmax**.

In [5]:
pred = model.predict(x_test)
pred

2/2 [==============================] - 0s 1ms/step


array([[8.5404426e-02, 4.6121892e-01, 4.5337668e-01],
       [9.9608684e-01, 1.8597472e-03, 2.0533870e-03],
       [8.5404426e-02, 4.6121892e-01, 4.5337668e-01],
       [8.5404426e-02, 4.6121892e-01, 4.5337668e-01],
       [8.5404426e-02, 4.6121892e-01, 4.5337668e-01],
       [9.9010921e-01, 4.7345213e-03, 5.1563000e-03],
       [1.1099717e-01, 4.4736475e-01, 4.4163814e-01],
       [8.5404426e-02, 4.6121892e-01, 4.5337668e-01],
       [8.5404426e-02, 4.6121892e-01, 4.5337668e-01],
       [8.5404426e-02, 4.6121892e-01, 4.5337668e-01],
       [8.5404426e-02, 4.6121892e-01, 4.5337668e-01],
       [9.8584425e-01, 6.7948229e-03, 7.3608030e-03],
       [9.9646997e-01, 1.6763047e-03, 1.8536601e-03],
       [9.8719805e-01, 6.1402558e-03, 6.6616768e-03],
       [9.9671811e-01, 1.5576334e-03, 1.7242834e-03],
       [8.5404426e-02, 4.6121892e-01, 4.5337668e-01],
       [8.5404426e-02, 4.6121892e-01, 4.5337668e-01],
       [8.5404426e-02, 4.6121892e-01, 4.5337668e-01],
       [8.5404426e-02, 4.612

In [6]:
pred = np.argmax(pred,axis=1) # raw probabilities to choose class (highest probability)
print(pred)

[1 0 1 1 1 0 1 1 1 1 1 0 0 0 0 1 1 1 1 1 0 1 0 1 1 1 1 1 0 0 0 0 1 0 0 1 1
 0]


Now that we have the actual iris flower predicted, we can calculate the percent accuracy (how many were correctly classified).

In [7]:
y_true= np.argmax(y_test,axis=1) 

score = metrics.accuracy_score(y_true, pred)

print("Accuracy score: {}".format(score))

Accuracy score: 0.6842105263157895


In [8]:
score = metrics.precision_score(y_true, pred, average= "weighted")
print("Precision score: {}".format(score))

Precision score: 0.5331807780320366


c:\Users\cdub6\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
score = metrics.recall_score(y_true, pred, average= "weighted")
print("Recall score: {}".format(score))

Recall score: 0.6842105263157895


In [10]:
score = metrics.f1_score(y_true, pred, average= "weighted")
print("F1 score: {}".format(score))

F1 score: 0.5820433436532507


In [11]:
print(metrics.classification_report(y_true, pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       0.48      1.00      0.65        11
           2       0.00      0.00      0.00        12

    accuracy                           0.68        38
   macro avg       0.49      0.67      0.55        38
weighted avg       0.53      0.68      0.58        38



c:\Users\cdub6\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\cdub6\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\cdub6\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### (2) Calculate Classification Cross-Entropy Loss (Log Loss)  

Log loss is an error metric that is often used in place of accuracy for classification.  

Log loss allows for "partial credit". For example, a model might be used to classify A, B and C.  The correct answer might be A, however if the classification network chose B as having the highest probability, then accuracy gives the neural network no credit for this classification.  

However, with log loss, the probability of the correct answer is added to the score.  For example, the correct answer might be A, but if the neural network only predicted .4 probability of A being correct, then the value -log(.4) is added.

$$ logloss = -\frac{1}{N}\sum^N_{i=1}\sum^M_{j=1}y_{ij} \log(\hat{y}_{ij}) $$

The following code shows the logloss scores that correspond to the average probablity for the correct item. The **pred** column specifies the average robability for the correct class.  The **logloss** column specifies the log loss for that probability.


Calculating log loss

In [12]:
# Generate predictions
pred = model.predict(x_test)

print("Numpy array of predictions")
print(pred[0:5])
print()
print("y_test:")
print(y_test[0:5])

score = metrics.log_loss(y_test, pred)
print("Log loss score: {}".format(score))

2/2 [==============================] - 0s 2ms/step
Numpy array of predictions
[[0.08540443 0.46121892 0.45337668]
 [0.99608684 0.00185975 0.00205339]
 [0.08540443 0.46121892 0.45337668]
 [0.08540443 0.46121892 0.45337668]
 [0.08540443 0.46121892 0.45337668]]

y_test:
[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]
Log loss score: 0.47844141222769554


## Evaluating Regression Models

Regression results are evaluated differently than classification.  Consider the following code. 

In [13]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn import metrics

path = "./data/"

filename_read = os.path.join(path,"auto-mpg.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])

cars = df['name']
df.drop('name',axis=1,inplace=True)
missing_median(df, 'horsepower')

encode_text_dummy(df, 'origin')

x,y = to_xy(df,"mpg")

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split (x, y, test_size=0.25, random_state=45)

model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], activation='relu'))
model.add(Dense(10))
model.add(Dense(10))
model.add(Dense(10))

model.add(Dense(1))  # 1 output neuron 


model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

model.fit(x_train,y_train, validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)


Epoch 1/1000
10/10 - 1s - loss: 152085.2188 - val_loss: 87293.7812 - 626ms/epoch - 63ms/step
Epoch 2/1000
10/10 - 0s - loss: 58907.1953 - val_loss: 30779.1270 - 27ms/epoch - 3ms/step
Epoch 3/1000
10/10 - 0s - loss: 19483.8633 - val_loss: 8845.9033 - 29ms/epoch - 3ms/step
Epoch 4/1000
10/10 - 0s - loss: 5080.9048 - val_loss: 1630.5200 - 28ms/epoch - 3ms/step
Epoch 5/1000
10/10 - 0s - loss: 817.3141 - val_loss: 209.7678 - 27ms/epoch - 3ms/step
Epoch 6/1000
10/10 - 0s - loss: 262.0631 - val_loss: 275.7950 - 27ms/epoch - 3ms/step
Epoch 7/1000
10/10 - 0s - loss: 355.9479 - val_loss: 300.5367 - 26ms/epoch - 3ms/step
Epoch 8/1000
10/10 - 0s - loss: 323.8880 - val_loss: 224.3014 - 27ms/epoch - 3ms/step
Epoch 9/1000
10/10 - 0s - loss: 257.8358 - val_loss: 181.3028 - 27ms/epoch - 3ms/step
Epoch 10/1000
10/10 - 0s - loss: 229.0906 - val_loss: 174.6464 - 28ms/epoch - 3ms/step
Epoch 11/1000
10/10 - 0s - loss: 227.4162 - val_loss: 173.7866 - 31ms/epoch - 3ms/step
Epoch 12/1000
10/10 - 0s - loss: 226

### Mean Square Error

The mean square error is the sum of the squared differences between the prediction ($\hat{y}$) and the expected ($y$).  MSE values are not of a particular unit.  If an MSE value has decreased for a model, that is good. Low MSE values are desired.

$ \text{MSE} = \frac{1}{n} \sum_{i=1}^n \left(\hat{y}_i - y_i\right)^2 $


In [14]:
# Predict
pred = model.predict(x_test)

# Measure MSE error.  
score = metrics.mean_squared_error(pred,y_test)
print("Final score (MSE): {}".format(score))

4/4 [==============================] - 0s 1ms/step
Final score (MSE): 49.599571228027344


### Root Mean Square Error

The root mean square (RMSE) is essentially the square root of the MSE.  Because of this, the RMSE error is in the same units as the training data outcome. Low RMSE values are desired.

$ \text{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^n \left(\hat{y}_i - y_i\right)^2} $

In [15]:
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

Final score (RMSE): 7.042696475982666


# Performance Improvement by Normalizing Features and Tuning Hyperparameters

There are many different settings that you can use for a neural network.  These can affect performance.  The following code changes some of these, beyond their default values:

* **activation:** relu, sigmoid, tanh
* **Layers and Neuron Counts**
* **optimizer:** adam, sgd, rmsprop, and [others](https://keras.io/optimizers/)

In [16]:
import pandas as pd
import io
import requests
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

path = "./data/"
preprocess = True

filename_read = os.path.join(path,"auto-mpg.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])

# create feature vector
missing_median(df, 'horsepower')
encode_text_dummy(df, 'origin')
df.drop('name',axis=1,inplace=True)

if preprocess:
    encode_numeric_zscore(df, 'horsepower')
    encode_numeric_zscore(df, 'weight')
    encode_numeric_zscore(df, 'cylinders')
    encode_numeric_zscore(df, 'displacement')
    encode_numeric_zscore(df, 'acceleration')
    encode_numeric_zscore(df, 'year')

# Encode to a 2D matrix for training
x,y = to_xy(df,'mpg')

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

model = Sequential()
model.add(Dense(100, input_dim=x.shape[1], activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=2, mode='auto')

model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)


Epoch 1/1000
10/10 - 1s - loss: 610.5696 - val_loss: 564.6489 - 545ms/epoch - 55ms/step
Epoch 2/1000
10/10 - 0s - loss: 582.4928 - val_loss: 530.4609 - 27ms/epoch - 3ms/step
Epoch 3/1000
10/10 - 0s - loss: 539.9799 - val_loss: 474.5724 - 27ms/epoch - 3ms/step
Epoch 4/1000
10/10 - 0s - loss: 468.2737 - val_loss: 385.7480 - 26ms/epoch - 3ms/step
Epoch 5/1000
10/10 - 0s - loss: 359.8221 - val_loss: 261.8332 - 27ms/epoch - 3ms/step
Epoch 6/1000
10/10 - 0s - loss: 215.6219 - val_loss: 125.5687 - 26ms/epoch - 3ms/step
Epoch 7/1000
10/10 - 0s - loss: 82.4229 - val_loss: 40.5240 - 26ms/epoch - 3ms/step
Epoch 8/1000
10/10 - 0s - loss: 34.4791 - val_loss: 40.3534 - 26ms/epoch - 3ms/step
Epoch 9/1000
10/10 - 0s - loss: 31.7587 - val_loss: 25.9118 - 27ms/epoch - 3ms/step
Epoch 10/1000
10/10 - 0s - loss: 23.5447 - val_loss: 20.7784 - 26ms/epoch - 3ms/step
Epoch 11/1000
10/10 - 0s - loss: 20.0469 - val_loss: 17.2537 - 25ms/epoch - 2ms/step
Epoch 12/1000
10/10 - 0s - loss: 17.3691 - val_loss: 15.7482

In [17]:
# Predict and measure RMSE
pred = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Score (RMSE): {}".format(score))

3/3 [==============================] - 0s 1ms/step
Score (RMSE): 2.1771719455718994


In [18]:
# print out prediction
df_y = pd.DataFrame(y_test, columns=['ground_truth'])
df_pred = pd.DataFrame(pred, columns=['predicted'])
result = pd.concat([df_y, df_pred],axis=1)
result

,ground_truth,predicted
0,33.0,33.464355
1,28.0,31.140566
2,19.0,20.188667
3,13.0,14.964059
4,14.0,13.275361
...,...,...
75,19.9,20.938141
76,17.5,17.550453
77,28.0,30.380283
78,29.0,30.011919
